
---
<big><big><big><big><big><big>Sieci neuronowe 2018/19</big></big></big></big></big></big>

---
<big><big><big><big><big>Najważniejsze modele sieci konwolucyjnych</big></big></big></big></big>

---

In [ ]:
# -*- coding: utf-8 -*-

import numpy as np
import pandas as pd

%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
from matplotlib.ticker import LinearLocator, FormatStrFormatter

plt.style.use("fivethirtyeight")

from bokeh.io import gridplot, output_file, show
from bokeh.plotting import figure, output_notebook
from bkcharts import Scatter

In [ ]:
output_notebook()

In [ ]:
sns.set(font_scale=2.0)

# Typowy model
<a href="https://cs.stanford.edu/people/karpathy/convnetjs/demo/cifar10.html" target="_blank">Demo typowego modelu CNN dla problemu CIFAR10 [Andrej Karpathy, Stanford]</a>

Architektura
  * input: 32 x 32 x 3
  * konw : [+2] 5 x 5 --> 16, ReLU
  * pool : 2 x 2 [+2]
  * konw : [+2] 5 x 5 --> 20, ReLU
  * pool : 2 x 2 [+2]
  * konw : [+2] 5 x 5 --> 20, ReLU
  * pool : 2 x 2 [+2]
  * softm: 10 klas wyjściowych
  
  

# ImageNet: Alex net
<img src="../nn_figures/imagenet.pdf" width="80%">[Krizhevsky, Sutskever, Hinton NIPS]
1. Konkurs ILSVRC'2010 (ImageNet Large-Scale Visual Recognition Challenge)
  * oryginalny zbiór danych ma ponad 22 miliony obrazów etykietowanych ponad 22 tysiącami klas
  * zbiór danych konkursu to 1.2 miliona obrazów z 1000 różnych klas
  * 50 tysięcy walidacyjnych, 150 tysięcy testowych
  * obrazy różnych rozdzielczości przeskalowane i wycięte do $256\times256$
    * w końcowym uczeniu użyte obrazy $224\times224$
  * dwie miary błędów
    * __top-1__ binarna miara prawidłowa/nieprawidłowa
    * __top-5__ że prawidłowa etykieta __jest/nie jest__ wśród 5-ciu zaproponowanych
  * problem uczony na danych __jedynie__ z odjętą średnią dla każdego piksela/kanału
2. wyniki
  * top-1 37.5\% błędów
  * top-5 17.0\% błędów
    * jedna z wersji osiągnęła 15.3\% w top-5
    * drugi najlepszy miał w top-5 26.2\% błędów

### Architektura
<img src="../nn_figures/imagenet.pdf" width="80%">[Krizhevsky, Sutskever, Hinton NIPS]
1. 8 warstw: 5 konwolucyjnych i 3 warstwowe
1. ostatnia warstwa jest softmaxem o wymiarze 1000 zwracającym szanse dla kazdej z klas
2. kolejne warstwy konwolucyjne
  * pierwsza $224\times224\times3$ ma 96 kerneli o wymiarach $11\times11\times3$ i stride $s=4$
    * stąd wyjście to $55\times55\times96$
    * a liczba parametrów to $11\times11\times3\times96\approx35K$
    * niestety$\ldots$ wielkość obrazu $224\times224$ nie pasuje do kernela $11\times11$
      * $\lfloor(224 - 11)/4\rfloor+1=\lfloor213/4\rfloor+1=53+1=54\ldots$
      * obraz powinien być $227\times227$
      * nikt nie wie jak to zrobił Alex Krizhevsky
  * druga na obrazie $55\times55\times48$ ma 256 kerneli $5\times5\times48$
    * __uwaga__ warstwy konwolucyjne zostały zaimplementowane na dwóch GPU (Nvidia GTX-580 z 3GB pamięci każda)
    * skutkuje to __podziałem__ kanałów na dwa procesory
    * kernele w warstwie 3 biorą wejście z __wszystkich__ kerneli warstwy 2
    * jednak kernele warstwy 4 biorą wejście __jedynie__ z kerneli na __tym samym__ GPU
    * to powoduje zmianę głębokości
    * ten schemat ulepszył wyniki po ok. 1.5\% dla uzytych błędów
  * trzecia warstwa konwolucji na __dwóch__ obrazach $27\times27\times128$ używa 384 kerneli $3\times3\times256$
  * czwarta ma 384 kernele $3\times3\times192$ na obrazie $13\times13\times192$
  * piąta ma 256 kerneli $3\times3\times192$ na obrazie $13\times13\times192$ dając dwa obrazy $13\times13\times128$
3. warstwy max-pooling użyte są po pierwszej, po drugiej i po piątej konwolucji
  * pooling $3\times3$ ze stridem $2$
  * wbrew zwyczajowi warstwy te są __nakładające__ się, co minimalnie zmniejsza wymiar
  * po poolingu pierwszym i drugim jest warstwa _normalizacyjna_
    * dość specyficzna normalizacja
    * obecnie _nie używane_, bo nie daje żadnych dodatkowych efektów
4. po piątej warstwie następują dwie warstwy po 4096 neuronów każda
  * warstwy też podzielone pomiedzy dwa GPU
5. na końcu $1000$ neuronów wyjściowych 
5. łącznie sieć ma ponad 60 milionów parametrów

## Ciekawe cechy
1. pierwsze _znane_ użycie ReLU
2. warstwy normalizacyjne już nie uzywane
3. silna augmentacja
4. dropout z $p=05$
  * ale tylko w ostatnich warstwach fully connected
  * dwukrotne wydłużenie czasu nauczania
5. batch o wielkosci $128$
6. uczenie SGD z momentum $0.9$
7. $lr=0.01$ zmniejszane o rząd wielkości po wykryciu obszaru plateau (ręcznie)
8. regularyzacja $L2$
9. uzyskany błąd dla pojedynczej sieci wynosił $18.2\%$ (błąd top-5)
10. ensemble siedmiu sieci CNN zmniejszał go do $15.4\%$

### Overfitting
1. w trakcie uczenia sieć overfitowała
  * mimo ponad miliona przykładów
2. augmentacja danych
  * z oryginalnych $256\times256$ wybrane losowo obrazy $224\times224$
  * także losowe poziome odbicia
  * obrazy generowane na bieżąco
  * 2048 razy więcej obrazów
    * to oczywiście __nie__ oznacza 2048-krotnego zwiększenia informacji
3. w trakcie testowania
  * wybrane obrazy z rogów
  * obraz centralny
  * odbicia poziome każdego z nich
  * uśrednione $10$ znalezionych predykcji
4. modyfikacja intensywności kanałów RGB
  * PCA na kanałach
  * do każdego kanału dodana wielokrotność znalezionej składowej głównej przemnożonej przez warstość własną przez wartość losową z rokładu normalnego
    * do każdego pksela $I_{xy}=[I_{xy}^R,I_{xy}^G,I_{xy}^B]$ dodana wartość $$[p_1,p_2,p_3][\alpha_1\lambda_1,\alpha_2\lambda_2,\alpha_3\lambda_3]^T$$ gdzie $p_i,\lambda_i$ to $i$-te wektory i składowe główne macierzy kowariancji $x\times3$ wartosci RGB
  * ten schemat minimalizuje wpływ oświetlenia
  * polepszył on wyniki top-1 o ponad 1\%

### Nauczone filtry
<img src="../nn_figures/imagenet-filters.pdf" width="100%">[Krizhevsky, Sutskever, Hinton NIPS'2012]
1. model nauczył się szeregu różnych filtrów
2. w warstwie pierwszej
  * GPU1 nauczył się filtrów praktycznie nie zawierających kolorów, za to posiadających cechy wykrywające krawędzie i zwroty
  * GPU2 wykrywał bloby kolorów
  * według autorów następowało to zawsze __bez względu__ na inicjalizację
3. filtry warstwy pierwszej przypominają filtry Gabora
  * to w pewien sposób przypomina rozpoznawanie w warstwach V1 i V2 mózgu

### Obrazy
<img src="../nn_figures/imagenet-images.pdf" width="100%">[Krizhevsky, Sutskever, Hinton NIPS]

1. po lewej obrazy i ich klasyfikacje przez model
2. po prawej obrazy testowe w pierwszej kolumnie i po pięć obrazów najbliższych w euklidesowej przestrzeni ostatniej warstwy ukrytej
3. usunięcie którejkolwiek warstwy środkowej powodowało spadek skuteczności o ponad 2%
  * głębokość gra rolę

CIFAR-10 recognition [cs232n.stanford.edu](http://cs231n.stanford.edu).

## ZFNet
1. w kolejnym ImageNet zwycięzcą była sieć ZFNet (Zeiler, Fergus)
2. architektura oparta na AlexNet z modyfikacjami wynikającymi z doświadczeń
  * konwolucje $11\times11$, $stride=4$ w pierwszej warstwie zbyt drastycznie redukowały przestrzeń
    * w ZFNet w pierwszej warstwie konwolucje $7\times7$ ze $stride=2$
  * liczba filtrów $384, 384, 256$ w warstwach $3, 4, 5$ była zbyt mała
    * w ZFNet więcej filtrów $512, 1024, 512$
3. błąd zredukowany do $14.8\%$ (też yop-5)

# VGG-16 (Simonyan, Zisserman, ImageNet'2014)
<img src="../nn_figures/vgg16.png" width="100%"> [Simonyan, Zisserman, 2014]
1. Krizhevsky et al. zwracali uwagę, że głębokość gra rolę
2. jednocześnie wydawało się, że konwolucje muszą być duże na początku, by obejmować większy semantycznie obszar


## Architektura
3. jeśli jednak weźmiemy __mały__ wymiar kernela, to przy małym paddingu można utrzymać wymiar
  * $k=3$, $p=\lfloor3/2\rfloor$ i wtedy $$o=i+2\lfloor k/2\rfloor-(k-1)=i+2\left\lfloor\frac{2n+1}{2}\right\rfloor-(2n+1-1)=i+n-n=i$$
  * to pozwala na dużą liczbę warstw
4. kernel $3\times3$ jest najmniejszym pozwalającym na wykrywanie krawędzi i pojęć lewy/prawy, góra/dół
5. wejście stałe $224\times224$ z odjętymi średnimi pikseli (jak w AlexNet)
6. architektura na rysunku ma 16 warstw konwolucyjnych (wszystkie $3\times3$) w blokach
  1. $3\times3\times64$, ReLU, $3\times3\times64$, ReLU, max-pool
  2. $3\times3\times128$, ReLU, $3\times3\times128$, ReLU, max-pool
  3. $3\times3\times256$, ReLU, $3\times3\times256$, ReLU, $3\times3\times256$, ReLU,  max-pool
  4. $3\times3\times256$, ReLU, $3\times3\times512$, ReLU, $3\times3\times512$, ReLU,  max-pool
  5. $3\times3\times256$, ReLU, $3\times3\times512$, ReLU, $3\times3\times512$, ReLU,  max-pool
  6. FC 4096, FC 4096, FC 1000, softmax
7. rozszerzona sieć z 19 warstwami
  1. $3\times3\times64$, ReLU, $3\times3\times64$, ReLU, max-pool
  2. $3\times3\times128$, ReLU, $3\times3\times128$, ReLU, max-pool
  3. $3\times3\times256$, ReLU, $3\times3\times256$, ReLU, $3\times3\times256$, ReLU,  max-pool
  4. $3\times3\times256$, ReLU, $3\times3\times512$, ReLU, $3\times3\times512$, ReLU, $3\times3\times512$, ReLU,  max-pool
  5. $3\times3\times256$, ReLU, $3\times3\times512$, ReLU, $3\times3\times512$, ReLU, $3\times3\times512$, ReLU, max-pool
  6. FC 4096, FC 4096, FC 1000, softmax
7. w jeszcze innej konfiguracji z 16 warstwami ostatnie konwolucje w dwóch ostatnich blokach (4. i 5.) były o wymiarach $1\times1$
  * konwolucje $1\times1$ w zasadzie nie wykorzystują/wprowadzają żadnej informacji o sąsiedztwie
  * jednak zwiększają nieliniowość __nie__ zmieniając wymiaru
8. Ale...
  * ten model (podobnie jak GoogleNet) był stworzony tuż przed powstaniem BatchNormalization
  * autorzy uczyli "sprytnie"
    * najpierw nauczyli model z 11-warstwami
    * potem dodawali pojedyncze warstwy pośrodku
  * podobnie GoogleNet dodał pośrednie funkcje kosztu
  
## Uczenie
1. sieć ma większą liczbę parametrów niż AlexNet 
  * ok. 140 milionów
  * większość pamięci jest przy przetwarzaniu w dwóch pierwszych warstwach konwolucyjnych
    * w pierwszej $224\times224\times64\approx3.2M$
    * ale parametrów w pierwszej tylko $3\times3\times3\times64=1728$
  * większość parametrów w ostatnch warstwach FC
    * razem $7\times7\times512\times4096+4096\times4096+4096\times1000\approx123.5M$
    * okazuje się, że zamiast tylu warstw FC _lepiej_ jest użyć _average pooling_ w ostatniej warstwie konwolucyjnych
      * ostatnia warstwa konwolucyjna daje blok $7\times7\times512$
      * można uzyskać warstwę $512$ liczb przez _uśrednienie_ po poziomicy $7\times7$ liczb
      * i pozbyć się warstw FC co działa względnie dobrze!
2. uczenie SGD z momentum rozpoczęło się ze współczynnikiem uczenia $0.01$ by być zmniejszonym gdy spadek na zbiorze walidacyjnym zanikł
3. uczenie zostało zatrzymane po 370 tysiącach iteracji
4. model był uczony na dwóch rozdzielczościach: 256 i 384 pikseli

## Wyniki
1. wersje 16- i 19-warstwowe osiągały do 24.8% w top-1 i 7.5% w top-5
2. wersja złożona z 2 modeli osiągała 23.7% oraz 6.8% (7.32% w złożonym modelu)
3. to wyniki porównywalne ze zwycięzcą GoogleNet, który miał 6.67%
4. a model vgg-16 jest znacznie prostszy...

# GoogleNet (Szegedy et al.)
<img src="../nn_figures/inception-naive.pdf" width="50%"><img src="../nn_figures/inception-reduction.pdf" width="50%">[Szegedy et al. 2015]
1. ten model rozwinął głębokość
2. cały model zbudowany jest z modułów __inception__
  * konwolucje mają wymiary $5\times5$, $3\times3$ i $1\times1$
  * wymiary podyktowane wygodą, by łatwiej się zgadzały granice
  * każdy moduł jest połączeniem wielu konwolucji wraz z poolingiem
    * ekstrakcja cech na wielu poziomach jednocześnie
    * $1\times1$ pozwala na konwolucje nie przestrzenną (spatial), ale tzw. __cross-channel__

## Naiwny blok inception
<img src="../nn_figures/inception-naive.pdf" width="50%">

* konwolucje mają wymiary $5\times5$, $3\times3$ i $1\times1$
* jakie są wymiary
* niech wejście będzie $28\times28\times256$
  * konwolucje $1\times1$ dają wyjście $28\times28\times128$ ($128$ to liczba kanałów)
  * konwolucje $3\times3$ (ze stride $1$ i padding $1$) dają $28\times28\times192$
  * konwolucje $5\times5$ (ze stride $1$ i padding $2$) dają $28\times28\times96$
  * pooling $3\times3$ daje $28\times28\times256$ 
    * pooling nie redukuje głębokości i ta zawsze tylko rośnieb
* wszystkie wyniki są konkatenowane _w głąb_
  * to daje razem $28\times28\times(128+192+96+256)=672$!
* w efekcie bardzo złożone obliczeniowo
  * $1\times1$ wymagają $28\times28\times128\times1\times1\times256$
  * $3\times3$ wymagają $28\times28\times192\times3\times3\times256$
  * $5\times5$ wymagają $28\times28\times96\times5\times5\times256$
  * razem __ponad__ $850$ milionów operacji dla jednego bloku inception
* problemem jest bardzo duża głębokość

## Rozszerzony blok inception
<img src="../nn_figures/inception-reduction.pdf" width="50%">

* wykorzystanie modułów bottleneck przez konwolucje $1\times1$
* konwolucje bottleneck $1\times1$ redukują głębokość do $64$ filtrów dając $28\times28\times64$
  * zarówno te po warstwie poprzedniej, jak i ta po pooling
* w ten sposób mamy
  * $1\times1$ dają wyjście $28\times28\times128$ ($128$ to liczba kanałów)
  * bottleneck $1\times1$ dają wyjście $28\times28\times64$
  * $3\times3$ (ze stride $1$ i padding $1$) dają $28\times28\times192$
  * bottleneck $1\times1$ dają wyjście $28\times28\times64$  
  * $5\times5$ (ze stride $1$ i padding $2$) dają $28\times28\times96$
  * pooling $3\times3$ daje $28\times28\times256$ 
  * bottleneck $1\times1$ dają wyjście $28\times28\times64$
* i na wyjściu tylko 480 filtrów
* a ile operacji?
  * bottleneck  $1\times1$ potrzebują $28\times28\times64\times1\times1\times256$
  * bottleneck  $1\times1$ potrzebują $28\times28\times64\times1\times1\times256$
  * $1\times1$ wymagają $28\times28\times128\times1\times1\times256$
  * $3\times3$ wymagają $28\times28\times192\times3\times3\times64$
  * $5\times5$ wymagają $28\times28\times96\times5\times5\times64$
  * bottleneck  $1\times1$ potrzebują $28\times28\times64\times1\times1\times256$
  * razem tylko ok. $360$ milionów operacji
* wykorzystanie bottleneck pozwala także redukować głębokość kolejnych warstw

3. dla zapewnienia inwariantności na translacje używamy powtarzalnych bloków
  * blok ma analizować korelacje w warstwie i grupować je z dużą korelacją
  * te tworzą elementy kolejnej warstwy
  * klastry skoncentrowane w jednym regionie mogą być łatwo przetwarzane przez konwolucje $1\times1$ dla zwiększenia nieliniowości
  * bardziej rozległe cechy muszą być pokryte przez kernele o większym wymiarze
  * dodatkowy pooling też daje zyski
  * tak wygląda _naiwny_ moduł inception
4. poważnym problemem tak poustawianych jeden na drugim modułów będzie coraz większe rozrzucenie przestrzenne cech wyższego rzędu
  * to sugeruje zwiększenie frakcji udziału konwolucji $3\times3$ i $5\times5$ na wyższych poziomach
  * a to staje się coraz bardziej kosztowne na wyższych poziomach
  * łączenie wyjścia poolingu oraz konwolucji spowoduje zwiększenie liczby wyjść z każdym krokiem, co będzie nieefektywne
6. moduły inception są ustawiane jeden po drugim
  * to pozwala na zwiększanie ich liczby __bez__ niekontrolowanego wzrostu złożoności

## GoogLeNet

<img align="right" src="../nn_figures/googlenet.pdf" width="70%">

1. sieć jest bardzo głęboka
  * propagacja gradientów wgłąb może być problemem
  * aby cały model dawał dobre wyniki, cechy generowane na niższych i środkowych warstwach powinny być dyskryminatywne
2. GoogLeNet dodaje klasyfikatory na niższych i środkowych poziomach
  * proste moduły average pool --> $1\times1$ konwolucje --> FC --> FC --> softmax
    * softmax ma tyle wyjść ile oryginalny model na końcu
  * powinny wzmacniać poprawne klasyfikacje na tych poziomach
  * zwiększać sygnał gradientu
    * przeciwdziałanie zanikaniu
    * dodawać regularyzację
    * w ogóle umożliwić uczenie w erze bez batchNormalization
3. te klasyfikatory
  * są dodane w środkowych modułach 4a i 4d
  * mają postać małych sieci konwolucyjnych
  * ich koszt jest dodawany do całkowitego zdyskontowane (zwykle ich koszty przemnożone przez 0.3)
  * usunięte w trakcie inferencji

## Liczba parametrów
1. AlexNet miał ok. 60 milionów
2. VGG19 osiągnął w ILSVR'2014 wynik minimalnie słabszy od GoogLeNet
3. VGG19 ma zaletę bardzo __prostej__ architektury ale kosztem olbrzymiej liczby parametrów - ok. 180 milionów
4. GoogLeNet ma tylko 5 milionów parametrów
  * powinna być lepsza generalizacja
  * mniej dodatkowych chwytów (augmentacja, itp)

# Dekonwolucje
1. dekonwolucje są w rzeczywistości transponowanymi konwolucjami
2. pozwalają odwrócić cykl
  * cześć konwolucyjna od obrazu do reprezentacji
  * część odwracająca reprezentację do postaci obrazu
  
<img src="../nn_figures/101-inp.jpg" width="45%"><img src="../nn_figures/101-out.jpg" width="45%">
<img src="../nn_figures/699-inp.jpg" width="45%"><img src="../
nn_figures/699-out.jpg" width="45%"> [za P. Garg]
  * konwolucyjna
    * Conv2d(16, 4) -> ReLU -> MaxPool(2) -> Conv2d(32, 5) -> ReLU -> MaxPool(2) -> Conv2d(64, 3) -> ReLU
  * dekonwolucyjna 
    * ConvTransp2d(32, 3) -> ReLU -> MaxUnPool(2) -> ConvTransp2d(16, 5) -> ReLU -> MaxUnPool(2) -> ConvTransp2d(3, 4) -> ReLU
3. pozwala na odszumianie oraz semantyczną segmentację
4. max-pool __nie jest__ operacją odwracalną
  * jak zrobić un-pooling?
    * zapamiętując indeksy
      * każdy max-pool zapamietuje __która__ z pozycji w kernelu (np. 2x2) miała najwyższą wartość
      * przy odwracaniu ta pozycja dostaje wartość wyjściową, a pozostałe 0
    * a jeśli indeksy __nie są__ dostępne?
      * niech architektura będzie architekturą autoenkodera
      * od wejścia do warstwy latent $Z$ jest sieć konwolucyjna
      * od latent do wyjściowej dekonwolucyjna
      * zadaniem jest __wylosować__ $z\in{}Z$ co nie podaje indeksów
      * najprostszym rozwiązaniem jest __upsampling__

# Network in Network
1. filtr konwolucji jest w rzeczywistości uogólnionym modelem liniowym (GLM)
  * poziom abstrakcji nie jest wysoki
  * konieczne jest tworzenie wielu filtrów
  * problem gdy dane dla pewnego wzorca (concept) leżą na bardziej skomlikowanej powierzchni niż półpłaszczyna
2. warstwa __mlpconv__ wykorzystuje nieliniowy detektor
<img src="../nn_figures/nin.pdf" width="80%">

  * filtr konwolucyjny wraz z nieliniową aktywacją, np. ReLU, oblicza $$f_{ijk}\max(w_K^Tx_{ij},0)$$
    * naturalnie obliczana jest liczba warstw zgodna z definicją MLP - może być głęboką siecią
  * podobnie mlpconv mapuje pewien obszar (patch) do wartości wyjsciowej w następnej warstwie
  * dzieje się tak przez wielowarstwowy perceptron z nielioniowymi aktywacjami
  * MLP jest __wspólny__ dla wszystkich lokalnych pól wejściowych
  * średnia ostatniej warstwy jest przekazywana jako wartość zaufania do obliczonych cech
  * to przypomina $maxout$ gdzie wybierane jest maksimum z wielu obliczonych wartości

## Konwolucje $1\times1$
1. nikt nam nie zabrania ustalenia wymiaru kernela na $k=1$, ale po co...?
2. konwolucja $1\times1$ oblicza __nieliniową__ funkcję na pojedynczej kolumnie pikseli obrazu
  * to może być moduł typu NiN
3. pozwala na efektywną redukcję wymiarowości, tzn. liczby kanałów
  1. 256 kanałów -- konwolucje 1x1 --> 64 kanały -- konwolucje 4x4 --> 256 kanałów
  2. 256 kanałów -- konwolucje 4x4 --> 256 kanałów
  * które rozwiązanie jest szybsze???
4. konwolucje przestrzenne na kanałach przetworzonych wcześniej przez $1\times1$ odpowiadają konwolucją na przestrzeniach osadzonych (__embedding__)
  * takie konwolucje są bardzo wydajne nie tracąc informacji
  * przetwarzania na embeddingach są bardzo popularne np. w NLP

# Densenet
<img src="../nn_figures/denseblock.pdf" width="80%"> [Huang, Liu, Weinberger, Maaten 2017]
1. wyjście z każdej warstwy (lub ich grupy) jest podłączone do __wszystkich późniejszych__ warstw
  * połączenie jako 
    * konkatenacja wszystkich poprzednich warstw
    * batch normalization
    * ReLU
    * konwolucja $3\times3$
2. ponieważ konkatenacja nie byłaby możliwa przy zmianie rozmiarów map warstwy są poddawana poolingowi w warstwach dostosowujących
  * batch normalization
  * konwolucje $1\times1$
  * $2\times2$ average pooling
3. przed konwolucjami $3\times3$ dodawane są konwolucje $1\times1$ dla zmniejszenia wymiarowości i poprawie efektywności
<img src="../nn_figures/densenet.png" width="100%"> [Huang, Liu, Weinberger, Maaten]
4. DenseNet z różnymi parametrami (wielkościami warstw, ich głębokościami, etc) osiągał (wg. metodologii takiej jak w AlexNet) na zbiorze ILSVRC'2012
  * top-1: 23.6%, 22.1%, 21.5%, 20.8%
  * top-5: 6.66%, 5.92%, 5.44%, 5.29%

# Squeezenet
<img src="../nn_figures/squeezenet.png" width="100%"> [Indola, Han, Moskewicz, Ashraf, Dally, Keutzer]
1. celem autorów było __zmniejszenie__ sieci CNN
  * łatwiej jest uczyć małe modele w równoległym środowisku
  * prostsza aktualizacja nauczonych modeli, np. automatyczna przez Internet
  * możliwość implementacji na FPGA
2. próba rozwiązania w Squeezenet wykorzystując
  * zamiana dużej liczby konwolucji $3\times3$ na konwolucje $1\times1$
  * zmniejszenie liczby kanałów wejściowych dla konwolucji $3\times3$
    * przez wykorzystanie warstw __squeeze__
  * downsampling (pooling) __późno__ w architekturze, by konwolucje miały większe mapy aktywacji, co prowadzi do niższych błędów
3. moduł __fire__
<img src="../nn_figures/squeezenet-fire.pdf" width="60%"> [Indola, Han, Moskewicz, Ashraf, Dally, Keutzer]
  * moduł zgniatający z konwolucji $1\times1$
  * przekazujący wyjście do konwolucji $1\times1$ i $3\times3$
  * pozwala na zmniejszenie liczby kanałów wejsciowych do konwolucji $3\times3$
4. końcowa warstwa __nie ma__ pełnych połączeń, a tylko _average pooling_ jak w NiN
5. metoda kompresji sieci zastosowana do
  * AlexNet
    * SVD zmniejsza sieć 5-krotnie i top-1 do 56%
    * Network Pruning 9-krotnie, top-1 do 57.2%, top-5 do 29.8%
    * Deep Compression zmniejsza AlexNet 35-krotnie bez zmiany poziomu błędu
  * SqueezeNet osiąga 50-krotną redukcję względem AlexNet __bez zmiany__ poziomów błędu
    * Deep Compression z 6-bitową kwantyzacją i 33% rzadkością daje model o wielkości ok. 0.47MB
      * ponad 500 razy mniejszy niż AlexNet!!!